# LANL Earthquakes Experiment
# Spectrogram Analysis

This notebook continues from the point where Spectrogram_librosa.ipynb created a directory of spectrograms made using librosa library.

TODO - external:
- look for regression that returns distribution rather than point estimator?
- go on to transformers.

TODO - NN:
- track errors (both train & valid) in multi-level training process (use learn.recorder.losses / val_losses?).
- made 81.6% classification error with mixup. 81.7% without mixup and becoming worse quicky (after the 2nd iteration!).
- go over regression NN in fastai course in lesson 3 notebook.

- if looks good - go on to resnet50.
- add external (non-raw-signal) features.
   - find a way to extract the output of 1 or 2 layers before the final output (instead of the final output itself).
   - pass to Zahar a way to generate this "pre-output" as features for his model.
   - if possible - also pass to Zahar a way to train a network to generate these features (to allow features training with the same CV).
   - an alternative to 2 sequential models would be a single model with additional input in advanced layers - read about custom itemlist in fastai.


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import seaborn as sns
import re
from pathlib import Path
import pickle as pkl
from time import time
from tqdm import tqdm, tnrange, tqdm_notebook
from pprint import pprint
import os, sys
from warnings import warn
import itertools
from sklearn.model_selection import KFold, StratifiedKFold
from fastai.vision import *
from fastai.metrics import error_rate#, mean_absolute_error

In [ ]:
# 0 = none; 1 = load; 2 = run.
RUN_CV = 1
RUN_CLASSIFICATION = 2
RUN_REGRESSION = 2

## Cross Validation

A validation fold consists of 20% of the segments, in batches of 40 sequential segments (after x6 augmentation).

In [ ]:
# Path consts
BASE = Path('../Data')
TRAIN = BASE/'train_spec2'

# NN consts
BS = 32 # 64 already causes OOM error in the regression
MIXUP = TRUE

# Data consts
SIZE = (187,147)
meta = pd.read_csv(TRAIN/'train_spec_meta.csv')
n_hops = len(meta)

In [ ]:
def plot_folds(meta, K_FOLDS=None, COLORS = ('k','b','r','g','y')):
    if K_FOLDS is None:
        K_FOLDS = len(np.unique(meta.fold))
    
    plt.figure(figsize=(10,6))
    for k in range(K_FOLDS):
        plt.plot(np.argwhere(meta.fold==k), meta.loc[meta.fold==k,'time'], COLORS[k]+'.', label=k)
    plt.xlabel('Segment\n(with x6 overlapping)')
    plt.ylabel('TTF')
    plt.title('Train Data Folding')
    plt.grid()
    plt.legend()
    
    plt.figure(figsize=(10,6))
    for k in range(K_FOLDS):
        sns.distplot(meta.loc[meta.fold==k,'time'], color=COLORS[k], hist=False, label=k)
    plt.xlabel('TTF')
    plt.title('Train Data Folding')
    plt.legend()
    plt.grid()

In [ ]:
# Validation configuration
validation_type = 'load' if RUN_CV<2 else 'CV'
valid_rate = 0.2
K_FOLDS = int(1/valid_rate)
batch_size = 40
np.random.seed(0)


# Create validation set
if validation_type == 'random':
    n_batches = n_hops // batch_size
    n_val_batches = int(valid_rate * n_batches)
    validation_batches = np.random.choice(range(n_batches), n_val_batches, replace=False)
    val_ids = np.concatenate([np.arange(i0*batch_size,(i0+1)*batch_size,dtype=int) for i0 in validation_batches])
    
elif validation_type == 'sequential':
    val_ids = np.array(range(int((1-valid_rate)*n_hops),n_hops))
    
elif validation_type == 'CV':
    meta['fold'] = 0
    hops = meta.loc[np.arange(0, n_hops-batch_size, batch_size)]
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True)
    for k, (train_index, valid_index) in enumerate(skf.split(hops['filename'], hops['class'])):
        val_ids = np.concatenate([np.arange(batch_size*i,batch_size*(i+1)) for i in valid_index])
        meta.loc[val_ids, 'fold'] = k
    meta.to_csv(TRAIN/'train_spec_meta.csv', index=False)


# Summary & save/load
if validation_type == 'load':
    # Note: all these variables are actually useful only for a single fold.
    # for a full CV, use instead the column 'fold' in meta.
    val_ids, val_set, train_mean, train_std = pkl.load(open(TRAIN/'validation_set.pkl','rb'))
else:
    val_set = set(meta.iloc[val_ids]['filename'])
    train_mean = np.mean([m for i,m in enumerate(meta['mean']) if i not in val_ids])
    train_std = np.sqrt(np.mean([m for i,m in enumerate(meta['var']) if i not in val_ids])) # not really std...
    pkl.dump((val_ids,val_set,train_mean,train_std), open(TRAIN/'validation_set.pkl','wb'))

if 'fold' in meta.columns:
    plot_folds(meta)
    
(val_ids[:5], train_mean, train_std)

In [ ]:
#meta = meta.head(1000)

## Classification

In [ ]:
def classification_data(meta, fold, verbose=1):
    validation_set = {nm: k for nm,k in zip(meta.filename,meta.fold)}
    
    data = ImageDataBunch.create_from_ll(
        (ImageList.from_df(meta[['filename','class']], path=TRAIN, folder=None, suffix='', cols=0)
                    .split_by_valid_func(lambda nm: validation_set[os.path.basename(nm)]==fold)
                    .label_from_df(label_delim=None, cols=1)),
        ds_tfms=[], size=SIZE, bs=BS
    ).normalize()
    # Note: normalize() uses by default self.stats = self.batch_stats() = [torch.mean,torch.std].
    # This should be good since we have very weird values and we wish to normalize them to nore conventional values.
    # RESNET34 kind of assumes a more specific input distribution (imagenet_stats),
    # but as far as we don't have convergence issues, we'll stick to this.
    
    classification_data_info(data, verbose)
    
    return data

def classification_data_info(data, verbose):
    if verbose >= 1:
        print(data)
        plt.figure(figsize=(8,5))
        sns.distplot(classes2times(data.label_list.train.y.classes), color='b', hist=False, label='Train')
        sns.distplot(classes2times(data.label_list.valid.y.classes), color='r', hist=False, label='Valid')
        plt.xlabel('TTF [s]')
        plt.legend()
        plt.grid()
    if verbose >= 2:
        data.show_batch(rows=3, figsize=(9,6))
    
def classes2times(cls):
    return np.array([class2time(c) for c in cls])

def class2time(c):
    t = (int(c[:2])+int(c[-2:]))/2
    return min(t,13)

In [ ]:
def classification_learn(data, lr_range=(3e-5,3e-4), cycles=(4,4), mixup=True,
                         save_label=None, load=False, predict=True, find_lr=True):
    # initialize
    learn = cnn_learner(data, models.resnet34, metrics=error_rate)
    if mixup:
        learn.mixup()
    
    # train
    if load:
        if load == 'freezed':
            learn.load(f'model_spec_class_resnet34_freezed_mxp{mixup:d}_'+save_label)
        elif load == 'unfreezed':
            learn.load(f'model_spec_class_resnet34_unfreezed_mxp{mixup:d}_'+save_label)
        else:
            raise ValueError("load must be either 'freezed' or 'unfreezed'.")
    
    else:
        # TODO track training errors (on both train & validation sets)
        if cycles[0] > 0:
            learn.fit_one_cycle(cycles[0])
            if save_label is not None:
                learn.save(f'model_spec_class_resnet34_freezed_mxp{mixup:d}_'+save_label)

        if find_lr:
            learn.lr_find()
            learn.recorder.plot()

        if cycles[1] > 0:
            learn.unfreeze()
            learn.fit_one_cycle(cycles[1], max_lr=slice(lr_range[0],lr_range[1]))
            if save_label is not None:
                learn.save(f'model_spec_class_resnet34_unfreezed_mxp{mixup:d}_'+save_label)
                
    # predict
    if predict:
        yv = np.array([str(learn.predict(x)[0]) for x in tqdm_notebook(learn.data.label_list.valid.x)], dtype='S8')
    else:
        yv = None
    
    return learn, yv

In [ ]:
def classification_interpretation(learn=None, interp=None, detailed=2):
    if interp is None:
        interp = ClassificationInterpretation.from_learner(learn)
    
    print('MAE:\t{0:.5f}'.format(confusion2mae(interp.confusion_matrix())))
    
    if detailed >= 1:
        interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
        
    if detailed >= 2:
        interp.plot_top_losses(9, figsize=(15,11), largest=False, heatmap = True)
        interp.plot_top_losses(9, figsize=(15,11), largest=False, heatmap = False)
        interp.plot_top_losses(9, figsize=(15,11), largest=True,  heatmap = False)
        interp.plot_top_losses(9, figsize=(15,11), largest=True,  heatmap = True)
        
    return interp

def confusion2mae(M, class_limits=np.array([0,1,2,3,4,5,6,7,8,10,12,16])):
    class_centers = (class_limits[1:]+class_limits[:-1])/2
    class_lengths = np.diff(class_limits)
    tot_err = 0
    n = 0
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            n += M[i,j]
            tot_err += M[i,j] * (np.abs(class_centers[i]-class_centers[j]) if i!=j else class_lengths[i]/4)
    return tot_err / n

In [ ]:
def analyze_errors(y0, y, tit, folds=None,
                   class_limits=np.array([-99,0,1,2,3,4,5,6,7,8,10,12,14,99]), COLORS=('k','b','r','g','y')):
    tit = '['+tit+']'
    
    print('Sizes:\t', y0.shape, y.shape)
    errs = y - y0
    print('MAE:\t', np.abs(errs).mean())
    
    if folds is not None:
        fig, axs = plt.subplots(1,2, figsize=(15,8))
        for k in range(len(np.unique(folds))):
            sns.distplot(errors[folds==k], color=COLORS[k], hist=False, label=k, ax=axs[0])
        axs[0].set_xlabel('Predicted TTF')
        axs[0].set_title(tit+' Predictions per Folder')
        axs[0].legend()
        axs[0].grid()
        sns.boxplot(x='Folder', y='Absolute Error [s]', data=pd.DataFrame({'Fold':folds, 'Absolute Error [s]':np.abs(errs)}),
                    showmeans=True, ax=axs[1])
        axs[1].set_title(tit)
    
    fig, axs = plt.subplots(1,2, figsize=(15,4))
    axs[0].plot(y, color='r', label='Model Predictions')
    axs[0].plot(y0, color='g', label='Ground Truth')
    axs[0].set_ylabel('TTF [s]')
    axs[0].set_title(tit)
    axs[0].grid()
    axs[0].legend()
    axs[1].plot(errs)
    axs[1].set_ylabel('Error [s]')
    axs[1].set_title(tit)
    axs[1].grid()
    plt.tight_layout()
    
    fig, axs = plt.subplots(1,2, figsize=(15,4))
    sns.distplot(y, color='r', ax=axs[0], label='Model Predictions')
    sns.distplot(y0, color='g', ax=axs[0], label='Ground Truth')
    axs[0].set_title(tit)
    axs[0].set_xlabel('TTF [s]')
    axs[0].grid()
    axs[0].legend()
    sns.distplot(errs, ax=axs[1])
    axs[1].set_title(tit)
    axs[1].set_xlabel('Error [s]')
    axs[1].grid()
    plt.tight_layout()
    
    fig, axs = plt.subplots(1,3, figsize=(15,4))
    b, a = polyfit(y0, y, 1)
    axs[0].plot(y0, a*y0+b, 'r-')
    axs[0].plot(y0, y, 'b,')
    axs[0].set_title(tit)
    axs[0].set_xlabel('True TTF')
    axs[0].set_ylabel('Predicted TTF')
    axs[0].grid()
    b, a = polyfit(y0, errs, 1)
    axs[1].plot(y0, a*y0+b, 'r-')
    axs[1].plot(y0, errs, 'b,')
    axs[1].set_title(tit)
    axs[1].set_xlabel('True TTF')
    axs[1].set_ylabel('Error')
    axs[1].grid()
    b, a = polyfit(y, errs, 1)
    axs[2].plot(y, a*y+b, 'r-')
    axs[2].plot(y, errs, 'b,')
    axs[2].set_title(tit)
    axs[2].set_xlabel('Predicted TTF')
    axs[2].set_ylabel('Error')
    axs[2].grid()
    plt.tight_layout()
    
    order = [f'{a:d}-{b:d}' for a,b in zip(class_limits[:-1],class_limits[1:])]
    cls0 = [[f'{a:d}-{b:d}' for a,b in zip(class_limits[:-1],class_limits[1:]) if a<=t<b][0] for t in y0]
    cls = [[f'{a:d}-{b:d}' for a,b in zip(class_limits[:-1],class_limits[1:]) if a<=t<b][0] for t in y]
    df = pd.DataFrame({'True TTF':cls0, 'Predicted TTF':cls, 'Predicted TTF numeric':y, 'Error':errs})
    
    # consider adding number of observations in annotation:
    # https://python-graph-gallery.com/58-show-number-of-observation-on-violinplot/
    fig, axs = plt.subplots(3,1, figsize=(15,12))
    sns.violinplot(x='True TTF', y='Predicted TTF numeric', data=df, order=order, scale='width', showmeans=True, ax=axs[0])
    axs[0].set_ylabel('Predicted TTF')
    axs[0].set_title(tit)
    axs[0].grid()
    sns.violinplot(x='True TTF', y='Error', data=df, order=order, scale='width', showmeans=True, ax=axs[1])
    axs[1].set_title(tit)
    axs[1].grid()
    sns.violinplot(x='Predicted TTF', y='Error', data=df, order=order, scale='width', showmeans=True, ax=axs[2])
    axs[2].set_title(tit)
    axs[2].grid()
    plt.tight_layout()

In [ ]:
if RUN_CLASSIFICATION:

    LOAD_ONLY = 'unfreezed' if RUN_CLASSIFICATION==1 else False

    K = len(np.unique(meta.fold))
    assert(K == K_FOLDS), 'Bad number of folders.'
    print(f'Number of folders:\t{K:d}')
    n_classes = len(np.unique(meta['class']))
    print(f'Number of classes:\t{n_classes:d}')

    YV0_C = meta['class']
    YV_C = np.zeros(len(meta), dtype='S8')
    confusions = [np.zeros((n_classes, n_classes), dtype=int)]
    for k in range(K):
        data_class = classification_data(meta, k, verbose = (2 if k==0 else 0))
        learn_class, yv_class = classification_learn(data_class, cycles=(4,3), mixup=MIXUP, save_label=f'{k:d}', load=LOAD_ONLY)
        YV_C[meta.fold==k] = yv_class
        try:
            interp_class = classification_interpretation(learn_class, detailed = (2 if k==0 else 0))
            confusions += [interp_class.confusion_matrix()]
        except:
            warn('Classification interpretation failed.')

    pkl.dump((YV_C, confusions), open(TRAIN/'classification_results.pkl','wb'))

In [ ]:
if RUN_CLASSIFICATION:
    analyze_errors(meta.time, classes2times(YV_C), 'Classification vs. Continuous Labels: Validation')

## Regression

Note: range of MAE should be between 2.0 (excellent) and 2.2 (quite bad), for CV that takes groups of 20 x3-augmented-segments (i.e. 7 original segments, or 1.05M samples) as homogeneous buckets.

In [ ]:
def regression_data(meta, fold, verbose=1):
    validation_set = {nm: k for nm,k in zip(meta.filename,meta.fold)}
    
    data = (PointsItemList.from_df(meta[['filename','time']], TRAIN)
            .split_by_valid_func(lambda nm: validation_set[os.path.basename(nm)]==fold)
            .label_from_df(cols='time', label_cls=FloatList)
            .transform(size=SIZE)
            .databunch()).normalize()
    data.batch_size = BS
    # Note: normalize() uses by default self.stats = self.batch_stats() = [torch.mean,torch.std].
    # This should be good since we have very weird values and we wish to normalize them to nore conventional values.
    # RESNET34 kind of assumes a more specific input distribution (imagenet_stats),
    # but as far as we don't have convergence issues, we'll stick to this.
    
    regression_data_info(data, verbose)
    
    return data

def regression_data_info(data, verbose):
    if verbose >= 1:
        print(data)
        plt.figure(figsize=(8,5))
        sns.distplot(data.label_list.train.y.items, color='b', hist=True, label='Train')
        sns.distplot(data.label_list.valid.y.items, color='r', hist=True, label='Valid')
        plt.xlabel('TTF')
        plt.legend()
        plt.grid()
    if verbose >= 2:
        data.show_batch(rows=3, figsize=(9,6))

In [ ]:
def regression_learn(data, lr_range=(3e-5,3e-4), cycles=(4,2), mixup=True,
                     save_label=None, load=False, predict=True, find_lr=True):
    
    # initialize
    learn = cnn_learner(data, models.resnet34, loss_func=mean_absolute_error, metrics=mean_absolute_error)
    learn.loss = F.l1_loss
    if mixup:
        learn.mixup()
    
    # train
    if load:
        if load == 'freezed':
            learn.load(f'model_spec_reg_resnet34_freezed_mxp{mixup:d}_'+save_label)
        elif load == 'unfreezed':
            learn.load(f'model_spec_reg_resnet34_unfreezed_mxp{mixup:d}_'+save_label)
        else:
            raise ValueError("load must be either 'freezed' or 'unfreezed'.")
        
    else:
        # TODO track training errors (on both train & validation sets)
        if cycles[0] > 0:
            learn.fit_one_cycle(cycles[0])
            if save_label is not None:
                learn.save(f'model_spec_reg_resnet34_freezed_mxp{mixup:d}_'+save_label)

        if find_lr:
            learn.lr_find()
            learn.recorder.plot()

        if cycles[1] > 0:
            learn.unfreeze()
            learn.fit_one_cycle(cycles[1], max_lr=slice(lr_range[0],lr_range[1]))
            if save_label is not None:
                learn.save(f'model_spec_reg_resnet34_unfreezed_mxp{mixup:d}_'+save_label)
    
    # predict
    if predict:
        out = learn.get_preds(learn.data)
        yt0 = out[1]
        yt = np.reshape(out[0], (out[0].shape[0],))
        yv0 = learn.data.label_list.valid.y.items
        yv = np.array([learn.predict(x)[0].data[0] for x in tqdm_notebook(learn.data.label_list.valid.x)])
        # ridiculously can't find any better way to get validation predictions. maybe try generalizing from this:
        # np.array(model.get_preds(ds_type=DatasetType.Valid)[0])
    else:
        yt0, yt, yv0, yv = (None for _ in range(4))
    
    return learn, yt0, yt, yv0, yv

def mean_absolute_error(pred:Tensor, targ:Tensor, reduction=None)->Rank0Tensor:
    "Mean absolute error between `pred` and `targ`."
    """ A copied version of FastAI function with additional 'reduction' argument,
    in sake of compatibility with up-to-date Pytorch. """
    pred,targ = flatten_check(pred,targ)
    errs = torch.abs(targ.float() - pred.float())
    if reduction is None or reduction=='mean':
        return errs.mean()
    if reduction=='none':
        return errs
    return errs.sum()

# class L1LossFlat(nn.L1Loss):
#     '''Mean Absolute Error Loss; seems that it is not necessary anymore, and F.l1_loss does fine.'''
#     def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
#         return super().forward(input.view(-1), target.view(-1))
# learn.loss = L1LossFlat

In [ ]:
if RUN_REGRESSION:

    LOAD_ONLY = 'unfreezed' if RUN_REGRESSION==1 else False

    K = len(np.unique(meta.fold))
    print(f'Number of folders:\t{K:d}')

    YV0 = meta.time
    YV = np.zeros(len(meta))
    for k in range(K):
        data = regression_data(meta, k, verbose = (2 if k==0 else 0))
        learn, yt0, yt, yv0, yv = regression_learn(data, cycles=(4,2), mixup=MIXUP, save_label=f'{k:d}', load=LOAD_ONLY)
        YV[meta.fold==k] = yv
    
    YV_crop = np.maximum(np.minimum(YV, 16.5), 0)

    pkl.dump((YV0, YV, YV_crop), open(TRAIN/'regression_results.pkl','wb'))

In [ ]:
if RUN_REGRESSION:
    analyze_errors(YV0, YV_crop, 'Regression: Validation')

In [ ]:
def class_vs_reg(y0, yr, yc, class_limits=np.array([-99,0,1,2,3,4,5,6,7,8,10,12,14,99])):
    
    yc0, yc = yc, classes2times(yc)
    errs_class = yc - y0
    errs_reg = yr - y0
    print('Samples:\t', y0.shape)
    print('MAE - Classification:\t', np.mean(np.abs(errs_class)))
    print('MAE - Regression:\t', np.mean(np.abs(errs_reg)))
    
    order = [f'{a:d}-{b:d}' for a,b in zip(class_limits[:-1],class_limits[1:])]
    cls0 = [[f'{a:d}-{b:d}' for a,b in zip(class_limits[:-1],class_limits[1:]) if a<=t<b][0] for t in y0]
    #cls = [[f'{a:d}-{b:d}' for a,b in zip(class_limits[:-1],class_limits[1:]) if a<=t<b][0] for t in y]
    df = pd.DataFrame({
        'True TTF': cls0+cls0,
        'Model': len(cls0)*['Classification'] + len(cls0)*['Regression'],
        'Error': np.concatenate((errs_class, errs_reg)),
        'Absolute Error': np.concatenate((np.abs(errs_class), np.abs(errs_reg)))
    })
    
    fig, axs = plt.subplots(2,1, figsize=(15,10))
    sns.violinplot(ax=axs[0], x="True TTF", y="Error", hue="Model", data=df,
                   palette="muted", split=True, scale="width", inner="quartile", showmeans=True)
    axs[0].grid()
    sns.violinplot(ax=axs[1], x="True TTF", y="Absolute Error", hue="Model", data=df,
                   palette="muted", split=True, scale="width", inner="quartile", showmeans=True)
    axs[1].grid()

In [ ]:
if RUN_CLASSIFICATION and RUN_REGRESSION:
    class_vs_reg(YV0, YV_crop, YV_C)